<a href="https://colab.research.google.com/github/mr-nudo/intelligent-tools/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO: Import the required libraries

In [26]:
# TODO
import os
import re
import numpy as np
import tensorflow as tf
import random
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Adding random seed

In [27]:
# Set environment variables
os.environ['PYTHONHASHSEED'] = str(25)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

# Set seed values
np.random.seed(25)
tf.random.set_seed(25)
random.seed(25)

# TODO: Read and Preprocess the dataset

In [28]:
path = '/content/alice.txt'
text = ""

# Load and preprocess the text
with open(path, 'r') as file:
    text = file.read()

text = re.sub(r'[^\w\s]', '', text.lower())
lines = text.split('\n')

In [29]:
print(len(text))

140269


# TODO: Using tokenizers

In [30]:
# TODO: Tokenize the text
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts([text])
word_index = tokenizer.word_index
total_words = len(word_index) + 1

In [31]:
print(total_words)

2751


# TODO: Feature Engineering

In [32]:
# TODO: Create input sequences
input_sequences = []

for line in lines:
    tokens = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(tokens) + 1):
        input_sequences.append(tokens[:i])

# Identify the maximum sequence length
max_sequence_length = max(len(seq) for seq in input_sequences)

# Pad sequences
padded_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length)

# Print the number of input sequences
# num_sequences = len(padded_sequences)
# print(num_sequences)

# Optionally, print the padded sequences and tokenizer word index
# print(padded_sequences)
# print(tokenizer.word_index)

In [33]:
print(len(input_sequences))

26410


# TODO: Storing features and labels

In [34]:
# TODO: Create predictors and labels
predictors = padded_sequences[:, :-1]
labels = padded_sequences[:, -1]

# One-hot encode the labels
labels = to_categorical(labels, num_classes=len(tokenizer.word_index) + 1)

# TODO: Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(predictors, labels, test_size=0.2, random_state=25)

# Print the sizes of the train and validation subsets
print("Training features size:", X_train.shape)
print("Validation features size:", X_val.shape)
print("Training labels size:", y_train.shape)
print("Validation labels size:", y_val.shape)

Training features size: (21128, 15)
Validation features size: (5282, 15)
Training labels size: (21128, 2751)
Validation labels size: (5282, 2751)


# TODO: Building our model

In [ ]:
# TODO: Build your model

# TODO: Model training

In [ ]:
# TODO: Train your model

# TODO: Visualising the Training and Validation Accuracies and Losses against the number of Epochs

In [ ]:
# TODO: Plotting the training and validation loss and accuracy


# TODO: Generate text

In [ ]:
# TODO: Function to generate text
def generate_text():
    # TODO

# Generate text
seed_text = "Forest is"
next_words = 10
generated_text = generate_text()
print(generated_text)